In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

In [3]:
!pip install xgboost

In [4]:
train = pd.read_csv('input/30-days-of-ml/train_folds.csv')
test = pd.read_csv('input/30-days-of-ml/test.csv')
sample_submission = pd.read_csv('input/30-days-of-ml/sample_submission.csv')

In [5]:
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kFold
0,1,B,B,B,C,B,B,A,E,C,...,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,0
1,2,B,B,A,A,B,D,A,F,A,...,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,2
2,3,A,A,A,C,B,D,A,D,A,...,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4
3,4,B,B,A,C,B,D,A,E,C,...,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,3
4,6,A,A,A,C,B,D,A,E,A,...,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,1


In [47]:
test.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,B,B,B,C,B,B,A,E,E,...,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
1,5,A,B,A,C,B,C,A,E,C,...,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
2,15,B,A,A,A,B,B,A,E,D,...,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
3,16,B,B,A,C,B,D,A,E,A,...,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
4,17,B,B,A,C,B,C,A,E,C,...,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412


In [48]:
sample_submission.head()

,id,target
0,0,0.5
1,5,0.5
2,15,0.5
3,16,0.5
4,17,0.5


In [17]:
train.columns

Index(['id', 'cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5',
       'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12',
       'cont13', 'target', 'kFold'],
      dtype='object')

In [10]:
y = train.target
train.drop(['target', 'id'], axis=1, inplace=True)
test.drop(['id'], axis=1, inplace=True)

train.head()

AttributeError: 'DataFrame' object has no attribute 'target'

In [17]:
train.head()
train.drop('kFold', axis=1)

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,C,B,B,A,E,C,N,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,A,A,B,D,A,F,A,O,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,A,C,B,D,A,D,A,F,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,A,C,B,D,A,E,C,K,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,A,C,B,D,A,E,A,N,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,B,B,A,A,B,D,A,E,A,I,...,0.307883,0.769792,0.450538,0.934360,1.005077,0.853726,0.422541,1.063463,0.697685,0.506404
299996,A,B,A,C,B,B,A,E,E,F,...,0.736713,0.528056,0.508502,0.358247,0.257825,0.433525,0.301015,0.268447,0.577055,0.823611
299997,B,B,A,C,B,C,A,E,G,F,...,0.277074,0.688747,0.372425,0.364936,0.383224,0.551825,0.661007,0.629606,0.714139,0.245732
299998,A,B,A,C,B,B,A,E,E,I,...,0.805963,0.344404,0.424243,0.382028,0.468819,0.351036,0.288768,0.611169,0.380254,0.332030


In [20]:
train.drop(['kFold'], axis=1, inplace=True)
train

KeyError: "['kFold'] not found in axis"

In [23]:

test

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,C,B,B,A,E,E,I,...,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
1,A,B,A,C,B,C,A,E,C,H,...,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
2,B,A,A,A,B,B,A,E,D,K,...,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
3,B,B,A,C,B,D,A,E,A,N,...,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
4,B,B,A,C,B,C,A,E,C,F,...,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,B,A,A,C,B,D,A,E,E,I,...,0.287454,0.543800,0.682378,1.028978,1.022741,0.683903,0.877273,0.532410,0.605397,0.884581
199996,B,A,A,C,B,B,A,E,C,F,...,0.794881,0.432778,0.389775,0.359871,0.550013,0.492082,0.202295,0.416875,0.406205,0.758665
199997,A,B,B,C,B,B,A,E,C,I,...,0.514487,0.060997,0.171741,0.317185,0.150340,0.122109,0.390524,0.334026,0.378987,0.839416
199998,A,A,A,C,B,D,A,D,A,F,...,0.286144,1.061710,0.819811,0.901241,0.555339,0.844315,0.894193,0.794102,0.844279,0.890473


In [25]:
print("Train shape: ", train.shape, "\nTest shape: ", test.shape)

Train shape:  (300000, 24) 
Test shape:  (200000, 24)


In [27]:
from sklearn.preprocessing import OrdinalEncoder
cat_cols = [col for col in train.columns if 'cat' in col]

X = train.copy()
X_test = test.copy()
enc = OrdinalEncoder()
X[cat_cols] = enc.fit_transform(train[cat_cols])
X_test[cat_cols] = enc.transform(test[cat_cols])
X.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,2.0,13.0,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,1.0,1.0,0.0,0.0,1.0,3.0,0.0,5.0,0.0,14.0,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,0.0,0.0,0.0,2.0,1.0,3.0,0.0,3.0,0.0,5.0,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,1.0,1.0,0.0,2.0,1.0,3.0,0.0,4.0,2.0,10.0,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,0.0,0.0,0.0,2.0,1.0,3.0,0.0,4.0,0.0,13.0,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


In [28]:
X_test.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,4.0,8.0,...,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
1,0.0,1.0,0.0,2.0,1.0,2.0,0.0,4.0,2.0,7.0,...,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,4.0,3.0,10.0,...,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
3,1.0,1.0,0.0,2.0,1.0,3.0,0.0,4.0,0.0,13.0,...,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
4,1.0,1.0,0.0,2.0,1.0,2.0,0.0,4.0,2.0,5.0,...,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412


In [30]:
X.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,2.0,13.0,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,1.0,1.0,0.0,0.0,1.0,3.0,0.0,5.0,0.0,14.0,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,0.0,0.0,0.0,2.0,1.0,3.0,0.0,3.0,0.0,5.0,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,1.0,1.0,0.0,2.0,1.0,3.0,0.0,4.0,2.0,10.0,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,0.0,0.0,0.0,2.0,1.0,3.0,0.0,4.0,0.0,13.0,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


In [44]:

X_stats = X.describe()
X_stats = X_stats.transpose()
X_stats

,count,mean,std,min,25%,50%,75%,max
cat0,300000.0,0.356233,0.478886,0.000000,0.000000,0.000000,1.000000,1.000000
cat1,300000.0,0.483920,0.499742,0.000000,0.000000,0.000000,1.000000,1.000000
cat2,300000.0,0.153713,0.360674,0.000000,0.000000,0.000000,0.000000,1.000000
cat3,300000.0,1.800953,0.632288,0.000000,2.000000,2.000000,2.000000,3.000000
cat4,300000.0,0.999400,0.150576,0.000000,1.000000,1.000000,1.000000,3.000000
cat5,300000.0,1.894157,0.981864,0.000000,1.000000,1.000000,3.000000,3.000000
cat6,300000.0,0.040960,0.280504,0.000000,0.000000,0.000000,0.000000,7.000000
cat7,300000.0,3.897103,0.567165,0.000000,4.000000,4.000000,4.000000,7.000000
cat8,300000.0,2.396427,1.835474,0.000000,0.000000,2.000000,4.000000,6.000000
cat9,300000.0,7.382510,3.419299,0.000000,5.000000,8.000000,10.000000,14.000000


In [45]:
#正規化

def norm(x):
    return (x - X_stats['mean']) / X_stats['std']

normed_X = norm(X)
normed_X_test = norm(X_test)

In [46]:
normed_X.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1.344300,1.032692,2.346401,0.314804,0.003985,-0.910673,-0.146023,0.181423,-0.215981,1.642877,...,0.495895,-0.482689,-1.460603,-1.036940,-0.454260,-0.950947,-0.926789,-0.429434,-0.696181,1.621788
1,1.344300,1.032692,-0.426183,-2.848311,0.003985,1.126269,-0.146023,1.944577,-1.305618,1.935335,...,-0.894730,0.098848,0.434768,-0.095470,0.402103,-0.612056,1.685964,2.020004,-0.940530,-0.193719
2,-0.743879,-0.968339,-0.426183,0.314804,0.003985,1.126269,-0.146023,-1.581731,-1.305618,-0.696783,...,-0.860485,0.613775,-0.438014,1.675291,0.236533,1.691036,1.071763,0.661690,0.189707,1.146326
3,1.344300,1.032692,-0.426183,0.314804,0.003985,1.126269,-0.146023,0.181423,-0.215981,0.765505,...,-0.862181,0.694265,-1.085978,0.897722,0.755096,0.458570,-0.501980,1.087255,0.184353,-1.020815
4,-0.743879,-0.968339,-0.426183,0.314804,0.003985,1.126269,-0.146023,0.181423,-1.305618,1.642877,...,-0.849984,0.773065,-0.222549,0.509136,0.775449,2.210109,2.056192,1.367099,0.530904,-1.154740


In [57]:
from tqdm import tqdm
import gc
from functools import reduce
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras

In [62]:
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

tf.random.set_seed(777)
NFOLDS = 6
skf = StratifiedKFold(n_splits=NFOLDS)
# folds = skf.split(X, cl)
# folds = list(folds)


def def_model():
#     inp = L.Input(name="inputs", shape=(n_in,))
    model = keras.Sequential([
        L.Dense(64, activation="relu", name="d1", input_shape=[len(X.keys())]), 
        L.Dense(64, activation="relu", name="d2") 
        L.Dense(1)
    ])
    
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    
#     preds = L.Dense(4, activation="linear", name="preds")(x)

    model.compile(loss="mse", optimizer="adam",
                  metrics=['mae', tf.keras.metrics.RootMeanSquaredError()])
    return model

net = def_model()
print(net.summary())

# oof = np.zeros(y.shape)
# nets = []
# for idx in range(NFOLDS):
#     print("FOLD:", idx)
#     tr_idx, val_idx = folds[idx]
#     ckpt = ModelCheckpoint(f"w{idx}.h5", monitor='val_loss', verbose=1, save_best_only=True,mode='min')
#     reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=3, min_lr=0.0005)
#     es = EarlyStopping(monitor='val_loss', patience=6)
#     reg = make_model(X.shape[1])
# #     reg.fit(X[tr_idx], y[tr_idx], epochs=10, batch_size=35_000, validation_data=(X[val_idx], y[val_idx]),
# #             verbose=1, callbacks=[ckpt, reduce_lr, es])
#     reg.load_weights(f"w{idx}.h5")
#     oof[val_idx] = reg.predict(X[val_idx], batch_size=50_000, verbose=1)
#     nets.append(reg)
#     gc.collect()
#     #
# #

# mae = mean_absolute_error(y, oof)
# rmse = np.sqrt(mean_squared_error(y_valid, oof_preds[valid_id]))
# print("mae:", mae)
# print("rmse:", rmse)


# del tr


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
d1 (Dense)                   (None, 64)                1600      
_________________________________________________________________
d2 (Dense)                   (None, 64)                4160      
Total params: 5,760
Trainable params: 5,760
Non-trainable params: 0
_________________________________________________________________
None


In [63]:
example_batch = normed_X[:10]
example_result = net.predict(example_batch)
example_result

array([[1.4967858 , 0.        , 0.31007132, 0.6060863 , 0.23749973,
        0.28751573, 0.20409411, 0.        , 0.1474778 , 0.        ,
        0.28664994, 0.        , 0.35190642, 0.        , 0.        ,
        0.81133133, 0.8445195 , 0.26256022, 0.11722787, 0.        ,
        0.        , 1.2508324 , 0.86593705, 0.28976172, 0.70687294,
        0.        , 0.7280912 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.3425996 ,
        0.        , 0.        , 0.7796953 , 0.6972081 , 0.06608918,
        0.6357151 , 0.6092821 , 0.        , 0.        , 0.05680601,
        0.        , 0.        , 0.08361353, 0.6136998 , 0.24954031,
        0.        , 0.        , 0.        , 0.09645025, 0.42243806,
        0.42766568, 0.        , 0.        , 0.2962186 , 0.        ,
        0.        , 0.        , 0.5843457 , 0.        ],
       [0.97369623, 0.09862398, 1.2549014 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0. 

In [64]:
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')
        
EPOCHS = 1000

history = net.fit(
normed_X, y,
epochs=EPOCHS, validation_split=0.2, verbose=0,
callbacks=[PrintDot()])


........................

KeyboardInterrupt: 